In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

theme = 'plotly_dark'


## Сервисные методы

### Фабрика функций $\varphi(t) = f(x - t \cdot d)$


In [2]:
def phi_factory(func, x, d):
    return lambda t: func(x - t * d)


### Поиск минимума методом золотого сечения


In [3]:
def golden_method(func):
    right = 0
    left = 5
    eps = 0.00001
    phi = (1 + np.sqrt(5)) / 2
    resphi = 2 - phi
    x1 = left + resphi * (right - left)
    x2 = right - resphi * (right - left)
    f1 = func(x1)
    f2 = func(x2)
    while abs(right - left) > eps:
        if f1 < f2:
            right = x2
            x2 = x1
            f2 = f1
            x1 = left + resphi * (right - left)
            f1 = func(x1)
        else:
            left = x1
            x1 = x2
            f1 = f2
            x2 = right - resphi * (right - left)
            f2 = func(x2)
    return (x1 + x2) / 2


### Получение данных для построения трехмерных графиков


In [4]:
def setup_3d_data(func, num):
    x = y = np.linspace(-1.5, 1.5, num=num)
    x = np.repeat(x, num)
    y = np.tile(y, num)
    z = []
    for i in range(num**2):
        z.append(func(np.array([x[i], y[i]])))
    x = np.reshape(x, (num, num))
    y = np.reshape(y, (num, num))
    z = np.reshape(z, (num, num))
    return x, y, z


## 1. Градиентный спуск с постоянным шагом


In [5]:
def gradient_descent(func, grad, x_0, eps1, eps2, max_iter, t=0.0001):
    x = x_0
    k = 0
    df = pd.DataFrame({"x": []})
    df.index.name = "i"
    df.name = "x"
    df.loc[len(df)] = [x]
    while True:
        if np.isnan(x).any():
            return x, k, df
        grad_x = grad(x)
        if np.linalg.norm(grad_x) < eps1 or k >= max_iter:
            return x, k, df
        x_next = x - t * grad_x
        df.loc[len(df)] = [x_next]
        k += 1
        if np.linalg.norm(x_next - x) < eps2 and abs(func(x_next) - func(x)) < eps2:
            return x_next, k, df
        x = x_next


## 2. Метод спуска с дроблением шага


In [6]:
def step_splitting_descent(func, grad, x_0, eps1, eps2, max_iter, _=0):
    x = x_0
    k = 0
    e = 0.1
    t = 1
    df = pd.DataFrame({"x": []})
    df.index.name = "i"
    df.name = "x"
    df.loc[len(df)] = [x]
    while True:
        if np.isnan(x).any():
            return x, k, df
        grad_x = grad(x)
        if np.linalg.norm(grad_x) < eps1 or k >= max_iter:
            return x, k, df
        phi = phi_factory(func, x, grad_x)
        while phi(t) > func(x) + t * e * (grad_x.T @ (-grad_x)):
            t = t / 2
        x_next = x - t * grad_x
        df.loc[len(df)] = [x_next]
        k += 1
        if np.linalg.norm(x_next - x) < eps2 and abs(func(x_next) - func(x)) < eps2:
            return x_next, k, df
        x = x_next


## 3. Метод наискорейшего спуска


In [7]:
def steepest_descent(func, grad, x_0, eps1, eps2, max_iter, _=0):
    x = x_0
    k = 0
    df = pd.DataFrame({"x": []})
    df.index.name = "i"
    df.name = "x"
    df.loc[len(df)] = [x]
    while True:
        if np.isnan(x).any():
            return x, k, df
        grad_x = grad(x)
        if np.linalg.norm(grad_x) < eps1 or k >= max_iter:
            return x, k, df
        phi = phi_factory(func, x, grad_x)
        t = golden_method(phi)
        x_next = x - t * grad_x
        df.loc[len(df)] = [x_next]
        k += 1
        if np.linalg.norm(x_next - x) < eps2 and abs(func(x_next) - func(x)) < eps2:
            return x_next, k, df
        x = x_next


## 4. Метод сопряженных градиентов


In [8]:
def conjugate_gradient(func, grad, x_0, eps1, eps2, max_iter, _=0):
    x = x_prev = x_0
    k = 0
    prev_good = False
    df = pd.DataFrame({"x": []})
    df.index.name = "i"
    df.name = "x"
    df.loc[len(df)] = [x]
    while True:
        if np.isnan(x).any():
            return x, k, df
        grad_x = grad(x)
        if np.linalg.norm(grad_x) < eps1 or k >= max_iter:
            return x, k, df
        if k == 0:
            d = - grad(x_0)
        else:
            beta = (np.linalg.norm(grad_x))**2 / (np.linalg.norm(grad(x_prev)))**2
            d = -grad_x + beta * d
        phi = phi_factory(func, x, -d)
        t = golden_method(phi)
        x_next = x + t * d
        df.loc[len(df)] = [x_next]
        k += 1
        if np.linalg.norm(x_next - x) < eps2 and abs(func(x_next) - func(x)) < eps2:
            if prev_good:
                return x_next, k, df
            prev_good = True
        else:
            prev_good = False
        x_prev = x
        x = x_next


## 5. Тестируемые квадратичные функции


In [9]:
def f_1(x):
    return x @ np.array(np.mat('2 0; 1 1')) @ x.T

def f_1_grad(x):
    a = np.array(np.mat('2 0; 1 1'))
    return (a + a.T) @ x.T

In [10]:
def f_2(x):
    return x @ np.array(np.mat('3299.599043 -36.367496; -36.367496 1.400957')) @ x.T

def f_2_grad(x):
    a = np.array(np.mat('3299.599043 -36.367496; -36.367496 1.400957'))
    return (a + a.T) @ x.T


## 6. Тест

### $f_1(x)$


In [11]:
x_3d, y_3d, z_3d = setup_3d_data(f_1, 30)
fig = go.Figure(
    data=go.Surface(x=x_3d, y=y_3d, z=z_3d, colorscale='Viridis'),
    layout=go.Layout(
        width=750,
        height=700,
        margin=dict(l=40, r=40, t=50, b=15),
        title='y=f1(x)',
        template=theme,
        scene=go.layout.Scene(
            aspectmode='cube',
            camera=dict(
                eye=dict(x=1.4, y=1.4, z=1.4)
            )
        )
    )
)
fig.show()


#### Сходимость градиентного спуска с постоянным шагом


In [12]:
steps = np.array([0.451, 0.2, 0.1, 0.0004])
grad_methods = []
for i in range(len(steps)):
    grad_method = gradient_descent(f_1, f_1_grad, np.ones(2), 0.001, 0.001, 50000, steps[i])
    grad_methods.append(grad_method)

In [13]:
ax_data = np.linspace(-1.5, 1.5, num=30)

fig = make_subplots(rows=2, cols=2, subplot_titles=steps, vertical_spacing=0.1)
df_f1 = pd.DataFrame({'Шаг':[], 'Результат':[], 'Количество итераций':[]}, dtype=object)

id = 0
for i in range(2):
    for j in range(2):
        df_f1.loc[len(df_f1.index)] = [steps[id], grad_methods[id][2]['x'].iat[-1], grad_methods[id][1]]
        df_f1_x = grad_methods[id][2]

        x, y = np.split(np.array(df_f1_x.x.values.tolist()), 2, axis=1)
        fig.add_trace(
            go.Contour(
                x=ax_data, 
                y=ax_data, 
                z=z_3d.T, 
                colorscale='Viridis',
                name=steps[id],
                hovertemplate = ' x: %{x} <br> y: %{y} <br> z: %{z} <extra></extra>'
            ),
            row=i+1, col=j+1
        )
        fig.add_trace(
            go.Scatter(
                x=x.ravel(),
                y=y.ravel(),
                mode='lines+markers',
                line_color='red',
                name='trajectory'
            ),
            row=i+1, col=j+1
        )
        fig.add_scatter(x=x[0], y=y[0], marker=dict(color='yellow', opacity=0.7), name='start', row=i+1, col=j+1)
        fig.add_scatter(x=[0], y=[0], marker=dict(color='white', opacity=0.7), name='true min', row=i+1, col=j+1)
        id += 1
fig.update_layout(height=750, width=800, margin=dict(l=75, r=40, t=70, b=15), template=theme, showlegend=False)
fig.show()
df_f1

,Шаг,Результат,Количество итераций
0,0.4510,"[-0.000458837818192093, -0.00019005684722484537]",853
1,0.2000,"[-0.0006776485314559995, 0.0016359882751999992]",15
2,0.1000,"[-0.0019566804446876336, 0.004724350952880088]",27
3,0.0004,"[0.18391937995090446, 0.43624584442908004]",767



#### Работа методов


In [14]:
methods = [gradient_descent, step_splitting_descent, steepest_descent, conjugate_gradient]
names = ['Спуск с постоянным шагом', 'Спуск с дроблением шага', 'Наискорейший спуск', 'Сопряженных градиентов']

fig = make_subplots(rows=2, cols=2, subplot_titles=names, vertical_spacing=0.1)
df_f1 = pd.DataFrame({'Метод':[], 'Результат':[], 'Количество итераций':[]}, dtype=object)

id = 0
for i in range(2):
    for j in range(2):
        data_all = methods[id](f_1, f_1_grad, np.array([0.5, 1]), 0.001, 0.001, 10000, 0.2)
        df_f1.loc[len(df_f1.index)] = [names[id], data_all[2]['x'].iat[-1], data_all[1]]
        data = data_all[2]
        x, y = np.split(np.array(data.x.values.tolist()), 2, axis=1)
        fig.add_trace(
            go.Contour(
                x=ax_data,
                y=ax_data,
                z=z_3d.T,
                colorscale='Viridis',
                name=names[id],
                hovertemplate = ' x: %{x} <br> y: %{y} <br> z: %{z} <extra></extra>'
            ),
            row=i+1, col=j+1
        )
        fig.add_trace(
            go.Scatter(
                x=x.ravel(),
                y=y.ravel(),
                mode='lines+markers',
                line_color='red',
                name='trajectory'
            ),
            row=i+1, col=j+1
        )
        fig.add_scatter(x=x[0], y=y[0], marker=dict(color='yellow', opacity=0.7), name='start', row=i+1, col=j+1)
        fig.add_scatter(x=[0], y=[0], marker=dict(color='white', opacity=0.7), name='true min', row=i+1, col=j+1)
        id += 1
fig.update_layout(height=750, width=800, margin=dict(l=75, r=40, t=70, b=15), template=theme, showlegend=False)
fig.show()
df_f1

,Метод,Результат,Количество итераций
0,Спуск с постоянным шагом,"[-0.0006263261888511996, 0.001512085179596799]",16
1,Спуск с дроблением шага,"[-0.00039534270763397217, 0.0009544417262077332]",13
2,Наискорейший спуск,"[-0.0001324877938975228, 0.00024014786376353532]",7
3,Сопряженных градиентов,"[-7.924663179892377e-06, -6.345182063216548e-06]",2



#### Работа при изменении начальной точки


In [15]:
methods = [gradient_descent, step_splitting_descent, steepest_descent, conjugate_gradient]
names = ['Спуск с постоянным шагом', 'Спуск с дроблением шага', 'Наискорейший спуск', 'Сопряженных градиентов']

fig = make_subplots(rows=2, cols=2, subplot_titles=names, vertical_spacing=0.1)
df_f1 = pd.DataFrame({'Метод':[], 'Результат':[], 'Количество итераций':[]}, dtype=object)

id = 0
for i in range(2):
    for j in range(2):
        data_all = methods[id](f_1, f_1_grad, np.array([0.5, -1]), 0.001, 0.001, 10000, 0.2)
        df_f1.loc[len(df_f1.index)] = [names[id], data_all[2]['x'].iat[-1], data_all[1]]
        data = data_all[2]
        x, y = np.split(np.array(data.x.values.tolist()), 2, axis=1)
        fig.add_trace(
            go.Contour(
                x=ax_data,
                y=ax_data,
                z=z_3d.T,
                colorscale='Viridis',
                name=names[id],
                hovertemplate = ' x: %{x} <br> y: %{y} <br> z: %{z} <extra></extra>'
            ),
            row=i+1, col=j+1
        )
        fig.add_trace(
            go.Scatter(
                x=x.ravel(),
                y=y.ravel(),
                mode='lines+markers',
                line_color='red',
                name='trajectory'
            ),
            row=i+1, col=j+1
        )
        fig.add_scatter(x=x[0], y=y[0], marker=dict(color='yellow', opacity=0.7), name='start', row=i+1, col=j+1)
        fig.add_scatter(x=[0], y=[0], marker=dict(color='white', opacity=0.7), name='true min', row=i+1, col=j+1)
        id += 1
fig.update_layout(height=750, width=800, margin=dict(l=75, r=40, t=70, b=15), template=theme, showlegend=False)
fig.show()
df_f1

,Метод,Результат,Количество итераций
0,Спуск с постоянным шагом,"[0.0006511068079718395, -0.0015719108863590388]",17
1,Спуск с дроблением шага,"[-0.0005502700805664062, 0.0013284683227539062]",11
2,Наискорейший спуск,"[-0.00014377760885522848, -0.00017969777764366...",5
3,Сопряженных градиентов,"[-1.3490060534332438e-06, 2.0092862682702384e-06]",2



### $f_2(x)$


In [16]:
x_3d, y_3d, z_3d = setup_3d_data(f_2, 30)
z_3d1 = z_3d
fig = go.Figure(
    data=go.Surface(x=x_3d, y=y_3d, z=z_3d, colorscale='Viridis'),
    layout=go.Layout(
        width=750,
        height=700,
        margin=dict(l=40, r=40, t=50, b=15),
        title='y=f2(x)',
        template=theme,
        scene=go.layout.Scene(
            aspectmode='cube',
            camera=dict(
                eye=dict(x=1.45, y=1.45, z=1.45)
            )
        )
    )
)
fig.show()


#### Сходимость градиентного спуска с постоянным шагом


In [17]:
steps = np.array([0.00005, 0.0001, 0.0002, 0.0004])
grad_methods = []
for i in range(len(steps)):
    grad_method = gradient_descent(f_2, f_2_grad, np.ones(2), 0.00001, 0.00007, 10000, steps[i])
    grad_methods.append(grad_method)

C:\Users\moroz\AppData\Local\Temp\ipykernel_11848\2223063066.py:6: RuntimeWarning:

overflow encountered in matmul

C:\Users\moroz\AppData\Local\Temp\ipykernel_11848\381251614.py:14: RuntimeWarning:

invalid value encountered in subtract



In [18]:
ax_data = np.linspace(-1.5, 1.5, num=30)

fig = make_subplots(rows=2, cols=2, subplot_titles=steps, vertical_spacing=0.1)
df_f2 = pd.DataFrame({'Шаг':[], 'Результат':[], 'Количество итераций':[]}, dtype=object)

id = 0
for i in range(2):
    for j in range(2):
        df_f2.loc[len(df_f2.index)] = [steps[id], grad_methods[id][2]['x'].iat[-1], grad_methods[id][1]]
        df_f2_x = grad_methods[id][2]

        x, y = np.split(np.array(df_f2_x.x.values.tolist()), 2, axis=1)
        fig.add_trace(
            go.Contour(
                x=ax_data, 
                y=ax_data, 
                z=z_3d1.T, 
                colorscale='Viridis',
                name=steps[id],
                hovertemplate = ' x: %{x} <br> y: %{y} <br> z: %{z} <extra></extra>'
            ),
            row=i+1, col=j+1
        )
        fig.add_trace(
            go.Scattergl(
                x=x.ravel()[::20],
                y=y.ravel()[::20],
                mode='lines+markers',
                line_color='red',
                name='trajectory'
            ),
            row=i+1, col=j+1
        )
        fig.add_scatter(x=x[0], y=y[0], marker=dict(color='yellow', opacity=0.7), name='start', row=i+1, col=j+1)
        fig.add_scatter(x=[0], y=[0], marker=dict(color='white', opacity=0.7), name='true min', row=i+1, col=j+1)
        id += 1
fig.update_layout(height=750, width=800, margin=dict(l=75, r=40, t=70, b=15), template=theme, showlegend=False)
fig.show()
df_f2

,Шаг,Результат,Количество итераций
0,0.00005,"[0.006521430530015953, 0.5915057913835104]",5359
1,0.00010,"[0.0038570945177185804, 0.3498455951716697]",5305
2,0.00020,"[0.0019283583909062573, 0.1749056669137359]",4385
3,0.00040,"[nan, -inf]",1420



#### Работа методов


In [19]:
methods = [gradient_descent, step_splitting_descent, steepest_descent, conjugate_gradient]
names = ['Спуск с постоянным шагом', 'Спуск с дроблением шага', 'Наискорейший спуск', 'Сопряженных градиентов']

fig = make_subplots(rows=2, cols=2, subplot_titles=names, vertical_spacing=0.1)
df_f2 = pd.DataFrame({'Метод':[], 'Результат':[], 'Количество итераций':[]}, dtype=object)

id = 0
for i in range(2):
    for j in range(2):
        data_all = methods[id](f_2, f_2_grad, np.ones(2), 0.00001, 0.00001, 10000, 0.0002)
        df_f2.loc[len(df_f2.index)] = [names[id], data_all[2]['x'].iat[-1], data_all[1]]
        data = data_all[2]
        x, y = np.split(np.array(data.x.values.tolist()), 2, axis=1)
        fig.add_trace(
            go.Contour(
                x=ax_data,
                y=ax_data,
                z=z_3d.T,
                colorscale='Viridis',
                name=names[id],
                hovertemplate = ' x: %{x} <br> y: %{y} <br> z: %{z} <extra></extra>'
            ),
            row=i+1, col=j+1
        )
        fig.add_trace(
            go.Scattergl(
                x=x.ravel(),
                y=y.ravel(),
                mode='lines+markers',
                line_color='red',
                name='trajectory'
            ),
            row=i+1, col=j+1
        )
        fig.add_scatter(x=x[0], y=y[0], marker=dict(color='yellow', opacity=0.7), name='start', row=i+1, col=j+1)
        fig.add_scatter(x=[0], y=[0], marker=dict(color='white', opacity=0.7), name='true min', row=i+1, col=j+1)
        id += 1
fig.update_layout(height=750, width=800, margin=dict(l=75, r=40, t=70, b=15), template=theme, showlegend=False)
fig.show()
df_f2

,Метод,Результат,Количество итераций
0,Спуск с постоянным шагом,"[0.0002754577256551792, 0.02498452437029489]",9249
1,Спуск с дроблением шага,"[0.00022562993768827054, 0.020465051990945022]",7985
2,Наискорейший спуск,"[5.737881303882956e-05, 0.005183328063839246]",814
3,Сопряженных градиентов,"[2.7340413514594096e-08, -4.549005564539075e-06]",10



#### Работа при изменении начальной точки


In [20]:
methods = [gradient_descent, step_splitting_descent, steepest_descent, conjugate_gradient]
names = ['Спуск с постоянным шагом', 'Спуск с дроблением шага', 'Наискорейший спуск', 'Сопряженных градиентов']

fig = make_subplots(rows=2, cols=2, subplot_titles=names, vertical_spacing=0.1)
df_f1 = pd.DataFrame({'Метод':[], 'Результат':[], 'Количество итераций':[]}, dtype=object)

id = 0
for i in range(2):
    for j in range(2):
        data_all = methods[id](f_2, f_2_grad, np.array([0.1, -1]), 0.00001, 0.00001, 10000, 0.0002)
        df_f1.loc[len(df_f1.index)] = [names[id], data_all[2]['x'].iat[-1], data_all[1]]
        data = data_all[2]
        x, y = np.split(np.array(data.x.values.tolist()), 2, axis=1)
        fig.add_trace(
            go.Contour(
                x=ax_data,
                y=ax_data,
                z=z_3d.T,
                colorscale='Viridis',
                name=names[id],
                hovertemplate = ' x: %{x} <br> y: %{y} <br> z: %{z} <extra></extra>'
            ),
            row=i+1, col=j+1
        )
        fig.add_trace(
            go.Scattergl(
                x=x.ravel(),
                y=y.ravel(),
                mode='lines+markers',
                line_color='red',
                name='trajectory'
            ),
            row=i+1, col=j+1
        )
        fig.add_scatter(x=x[0], y=y[0], marker=dict(color='yellow', opacity=0.7), name='start', row=i+1, col=j+1)
        fig.add_scatter(x=[0], y=[0], marker=dict(color='white', opacity=0.7), name='true min', row=i+1, col=j+1)
        id += 1
fig.update_layout(height=750, width=800, margin=dict(l=75, r=40, t=70, b=15), template=theme, showlegend=False)
fig.show()
df_f1

,Метод,Результат,Количество итераций
0,Спуск с постоянным шагом,"[-0.00027543967957152963, -0.024982887557184422]",9219
1,Спуск с дроблением шага,"[-0.00022566200123346896, -0.02046796021370226]",7960
2,Наискорейший спуск,"[-0.00011517796104055683, -0.010560562952003496]",3526
3,Сопряженных градиентов,"[-8.774673683498247e-07, -7.618136513507304e-05]",10



## 7. Генератор функций


In [21]:
def f_gen(n, k):
    u, _ = np.linalg.qr(np.random.randn(n, n))
    d = np.diag(np.linspace(1, k, n))
    a = u @ d @ u.T
    return lambda x: x @ a @ x.T, lambda x: (a + a.T) @ x.T


## 8. Определение зависимости числа итераций $T$



### Зависимость $T$ от $n$ при $k = 2$ и от $k$ при $n = 2$


In [22]:
def t_all(n, k, method):
    res = []
    resx = []
    for i in range(len(n)):
        f, g = f_gen(n[i], k[i])
        x_0 = stats.uniform.rvs(size=n[i])
        data = method(f, g, x_0, 0.001, 0.001, 10000, 0.0001)
        res.append(data[1])
        resx.append(data[2]['x'].iat[-1])
    return np.array(res, dtype=object), np.array(resx, dtype=object)

In [23]:
x_nk = y_nk = np.unique(np.geomspace(1, 1000, num=20, dtype=int))
res_n, _ = t_all(x_nk, np.full(len(x_nk), 2), conjugate_gradient)
res_k, res_xk = t_all(np.full(len(y_nk), 2), y_nk, conjugate_gradient)

In [24]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('T(n, k=2)', 'T(n=2, k)'))
fig.add_scatter(
    x=x_nk,
    y=res_n,
    hovertemplate = 'Количество переменных: %{x} <br>Итераций: %{y} <extra></extra>',
    row=1, col=1
)
fig.add_scatter(
    x=y_nk,
    y=res_k,
    hovertemplate = 'Число обусловленности: %{x} <br>Итераций: %{y} <extra></extra>',
    row=1, col=2
)
fig.update_layout(height=400, width=1200, margin=dict(l=75, r=40, t=40, b=20), template=theme, showlegend=False, )
fig.update_layout(xaxis_title='n', yaxis_title='iters')
fig.update_layout(xaxis2_title='k', yaxis2_title='iters')
fig.show()

x, y = np.split(res_xk, 2, axis=1)

fig = go.Figure(
    data=go.Scatter(
        x=x.ravel(),
        y=y.ravel(),
        mode='markers',
        hovertemplate = ' x: %{x} <br> y: %{y} <extra></extra>',
    ),
    layout=go.Layout(
        title='Найденные минимумы',
        height=450,
        width=700,
        margin=dict(l=85, r=40, t=70, b=20), 
        xaxis=dict(tickformat='.5f', title='x'), 
        yaxis=dict(tickformat='.5f', title='y'),
        template=theme
    )
)
fig.show()


### Зависимость $T$ от обоих параметров $n, k$


In [25]:
xp = yp = np.unique(np.geomspace(1, 1000, num=20, dtype=int))
n = len(xp)
xp1 = np.repeat(xp, n)
yp1 = np.tile(yp, n)
zp1, _ = t_all(xp1, yp1, conjugate_gradient)
x = np.reshape(xp1, (n, n))
y = np.reshape(yp1, (n, n))
z = np.reshape(zp1, (n, n))

In [29]:
fig = go.Figure(
    data=go.Surface(x=x, y=y, z=z, colorscale='Viridis'),
    layout=go.Layout(
        width=750,
        height=750,
        margin=dict(l=40, r=40, t=50, b=15),
        title='T(n, k)',
        template=theme,
        scene=go.layout.Scene(
            aspectmode='cube',
            camera=dict(
                eye=dict(x=-1.5, y=-1.5, z=1.3)
            ),
            xaxis_title='n',
            yaxis_title='k',
            zaxis_title='iters'
        )
    )
)
fig.show()